In [10]:
import torch
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple
import torch
import torch.optim as optim
from model import Net
model = torch.load('model_weights/modelentire.pth')
model.load_state_dict(torch.load('model_weights/model.pth'))
loss_function = nn.MSELoss()
model.eval()

Net(
  (fc1): Linear(in_features=6, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
)

In [2]:
def weighted_mse(u_exp: np.ndarray, u: np.ndarray, time: np.ndarray, eps: float = + 10 ** (-15)) -> Tuple[np.ndarray, np.ndarray]:
    dt = np.diff(time, n=1, axis=0)
    dt = np.append(dt, dt[-1])
    amplitude = np.dot(dt, u * u_exp) / np.dot(dt, u ** 2)
    e = (u_exp - amplitude * u) / (u_exp + eps) * dt
    
    loss = np.dot(e, e)
#     if (np.log(loss) == math.inf):
#         print('u_exp\n', u_exp)
#         print('u\n', u)
#         print('amplitude = ', amplitude)
#         print ('loss = ', loss)
#         sys.exit()
    return loss, amplitude, e



t = 1e-3 * np.array([
    0.135, 0.180, 0.235, 0.315, 0.365, 0.420,
    0.485, 0.560, 0.650, 0.750, 0.865, 1.000,
    1.155, 1.335, 1.540, 1.780, 2.055, 2.370,
    2.740, 3.160, 3.650, 4.215, 4.870, 5.625,
    6.495, 7.500, 8.660, 10.00, 11.55, 13.34,
    15.40, 17.79, 20.54, 23.72, 27.39, 31.63,
    36.52, 42.17, 48.70, 56.24, 64.94, 74.99,
    86.60, 100.0, 115.5, 133.4
])

In [3]:
df = pd.read_pickle('Dataset')
df

,Sigma,mu,d,th,th0,Decay
19628,-1.271194,-0.747548,0.0,-1.085085,0.0,"[0.19375629692752577, 0.18294351872284478, 0.1..."
2486,-0.522897,0.229992,0.0,-0.732977,0.0,"[0.192993357268207, 0.1820247479384618, 0.1711..."
64883,1.436271,-0.737692,0.0,0.091825,0.0,"[0.1752133606610032, 0.16091191549296152, 0.14..."
10027,1.629299,1.329376,0.0,-1.526841,0.0,"[0.18759459262951242, 0.17560025481240513, 0.1..."
51126,0.423736,-0.777624,0.0,-0.247653,0.0,"[0.17945400597800243, 0.1659367937378173, 0.15..."
...,...,...,...,...,...,...
69892,0.814175,0.781662,0.0,-0.878862,0.0,"[0.1882601086365116, 0.17643418646482018, 0.16..."
72626,-0.020505,-0.181898,0.0,-0.651101,0.0,"[0.18716586487573802, 0.17512817120621807, 0.1..."
17249,0.133407,1.380844,0.0,0.863656,0.0,"[0.1928662308439771, 0.18225947646392704, 0.17..."
38159,1.210575,-0.099357,0.0,1.416446,0.0,"[0.1803007929163741, 0.16759557921691812, 0.15..."


In [35]:
# i = 0
# graph_df = df.iloc[0:1000]
# fix_str = graph_df.sample(1)
# #fix_vector = np.array([fix_str.get('Sigma').iloc[0], fix_str.get('mu').iloc[0], fix_str.get('th').iloc[0]])
# x = np.array([])
# y = np.array([])
# z = np.array([])
# r = np.array([])
# for _,row in graph_df.iterrows():
#     norm_2d = np.linalg.norm([fix_str.get('Sigma').iloc[0] - row['Sigma'],
#                             fix_str.get('mu').iloc[0] - row['mu'],
#                               fix_str.get('th').iloc[0] - row['th']])
#     #the_point = [row['Sigma'],row['mu'],row['th'],norm_2d]
#     #data_for_plotting.append(the_point)
#     x = np.append(x,row.get('Sigma'))
#     y = np.append(y, row.get('mu'))
#     z = np.append(z, row.get('th'))
#     r = np.append(r,norm_2d)
# print(len(x))

1000


In [41]:
# x = x
# X,Y = np.meshgrid(x,y)
# Z,Y = np.meshgrid(z,y)
# V = np.sin(r)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# #ax.view_init(45,60)
# ax.plot_surface(X, Y, Z, facecolors=cm.Oranges(V))

In [413]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

device = get_device()
net = model.double().to(device)

# def appending_to_tensor(tensor, fix_vector, variable_vector):
#     tmp_tensor = torch.tensor([rand_2_strs.get('Sigma').iloc[0],rand_2_strs.get('Sigma').iloc[1],
#                                rand_2_strs.get('mu').iloc[0],rand_2_strs.get('mu').iloc[1],
#                                rand_2_strs.get('th').iloc[0],rand_2_strs.get('th').iloc[1],
#                                np.log(weighted_mse(np.array(rand_2_strs.get('Decay').iloc[0]),np.array(rand_2_strs.get('Decay').iloc[1]),t)[0])])
#     tmp_tensor = torch.unsqueeze(tmp_tensor, 0)
#     tmp_tensor = torch.cat((tensor,tmp_tensor),0)
#     return(tmp_tensor)

def mean_absolute_percentage_error(y_pred, y_true, eps=10**(-2)): 
    return abs(torch.mean(((y_true-y_pred))/(y_true + eps)))*100


# def create_batches_to_device(df, device, num_fix_string,batch_size=1):
#     tensor = torch.tensor([])
#     fix_vector = 
#     variable_vector = 
#     tensor = appending_to_tensor(tensor, rand_train_str)
#     tensor = tensor.to(device)
#     trainset = torch.utils.data.DataLoader(tensor, batch_size=batch_size, shuffle=True)
#     return (trainset)
print(device)


def l2_norm(variable_vector):
    return np.linalg.norm([fix_vector.get('Sigma').iloc[0] - variable_vector[0],
                          fix_vector.get('mu').iloc[0] - variable_vector[1],
                          fix_vector.get('th').iloc[0] - variable_vector[2]])

def L(variable_vector):
    tensor = torch.tensor([fix_vector.get('Sigma').iloc[0],variable_vector[0],
                           fix_vector.get('mu').iloc[0],variable_vector[1],
                           fix_vector.get('th').iloc[0],variable_vector[2]])
    tensor = tensor.to(device)
    #print(tensor)
    #data_batch = torch.utils.data.DataLoader(tensor, batch_size=1, shuffle=False)
    #print(data_batch)
#     for data in data_batch:
#         X = data
#         print(X)
#         output = net(X.double())
#         print (output)
    output = net(tensor.double())
    #print (output.item())
    return (np.exp(output.item()))

constant = 1

def minimization_func(variable_vector):
    #variable_vector = [Sigma2, mu2, th2]
    return (-1 * l2_norm(variable_vector) + constant * L(variable_vector) )

print ('одинаковые параметры loss = ', L([fix_vector.get('Sigma').iloc[0],fix_vector.get('mu').iloc[0],fix_vector.get('th').iloc[0]]))
print ('одинаковые параметры l2_norm =', l2_norm([fix_vector.get('Sigma').iloc[0],fix_vector.get('mu').iloc[0],fix_vector.get('th').iloc[0]]))
print ('одинаковые параметры minim = ', minimization_func([fix_vector.get('Sigma').iloc[0],fix_vector.get('mu').iloc[0],fix_vector.get('th').iloc[0]]))

cpu
одинаковые параметры loss =  0.00015044061624062795
одинаковые параметры l2_norm = 0.0
одинаковые параметры minim =  0.00015044061624062795


In [527]:
#fix_vector = df.sample(1) временно
fix_vector = df.loc[9030]
fix_vector = pd.DataFrame(fix_vector).transpose()
print ('Зафиксировали произвольный вектор из датафрейма с компонентами \n')
fix_vector

Зафиксировали произвольный вектор из датафрейма с компонентами 



,Sigma,mu,d,th,th0,Decay
9030,-0.809753,-1.56861,0,-0.248755,0,"[0.17439216402987712, 0.15983478220067351, 0.1..."


### Нахождение границ для оптимизатора с учетом параметров усреднения первоначальной выборки

In [528]:
means = np.fromfile('means_scaler')
print(means)
stds = np.fromfile('stds_scaler')
print(stds)
a = [[stds[0]]]
b = [[1e+06-means[0]]]
sigma_min = np.linalg.solve(a,b)
print('Sigma lower bound',sigma_min)
a = [[stds[1]]]
b = [[10-means[1]]]
mu_min = np.linalg.solve(a,b)
print ('mu lower bound', mu_min)
a = [[stds[3]]]
b = [[0.650209-means[3]]]
th_min = np.linalg.solve(a,b)
print ('th lower bound', th_min)

a = [[stds[0]]]
b = [[10e+06-means[0]]]
sigma_max = np.linalg.solve(a,b)
print('Sigma upper bound',sigma_max)
a = [[stds[1]]]
b = [[150-means[1]]]
mu_max = np.linalg.solve(a,b)
print ('mu upper bound', mu_max)
a = [[stds[3]]]
b = [[11.7-means[3]]]
th_max = np.linalg.solve(a,b)
print ('th upper bound', th_max)

[5.49793364e+06 8.00354882e+01 8.90000000e+01 6.18313815e+00
 6.50000000e+00]
[2.58728391e+06 4.04162034e+01 1.00000000e+00 3.19023924e+00
 1.00000000e+00]
Sigma lower bound [[-1.73847703]]
mu lower bound [[-1.73285668]]
th lower bound [[-1.73433048]]
Sigma upper bound [[1.74007435]]
mu upper bound [[1.73110055]]
th upper bound [[1.72929409]]


## Минимизация логарифмических величин

In [542]:
constant = 1000
from scipy.optimize import minimize
x0 = np.array([fix_vector.get('Sigma').iloc[0],fix_vector.get('mu').iloc[0],fix_vector.get('th').iloc[0]])
#res = minimize(minimization_func, x0, method='nelder-mead')
# res = minimize(minimization_func, x0, method='BFGS',options={'disp': True, 'eps':1e-6, 'gtol':1e1})
#res = minimize(minimization_func, x0, method='BFGS',options={'disp': True})
#res = minimize(minimization_func, x0, method='CG',options={'disp': True, 'maxiter':100000})
#res = minimize(minimization_func, x0, method='powell')
res =  minimize(minimization_func, x0, method='SLSQP', bounds=((sigma_min[0][0], sigma_max[0][0]),(mu_min[0][0], mu_max[0][0]),(th_min[0][0],th_max[0][0])),options={'disp': True})
#print(res)
print ('Start vector', [fix_vector.get('Sigma').iloc[0],fix_vector.get('mu').iloc[0],fix_vector.get('th').iloc[0]])
print ('Vector after minimization', res.x)
print ('Function value with start vector', minimization_func(x0))
print ('How % change loss', mean_absolute_percentage_error(torch.tensor(L(res.x)), torch.tensor(L(x0))).item())
print ('L2 norm with minim vector', l2_norm(res.x))

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2804286772740854
            Iterations: 12
            Function evaluations: 71
            Gradient evaluations: 12
Start vector [-0.8097534630349149, -1.5686084526495365, -0.24875458646855458]
Vector after minimization [-1.46126883  1.73110055 -1.20922668]
Function value with start vector 0.7328583265480415
How % change loss 4.514900098825052
L2 norm with minim vector 3.497864835014197


## Проверяю нейросеть на сходимостьь

In [509]:
i = total_iter = 10
fix_vector = df.sample(1)
while(i >= 0):
    variable_vector = [fix_vector.get('Sigma').iloc[0] + (i/total_iter * fix_vector.get('Sigma').iloc[0]) ,fix_vector.get('mu').iloc[0] + (i/total_iter * fix_vector.get('mu').iloc[0]),fix_vector.get('th').iloc[0] + (i/total_iter * fix_vector.get('th').iloc[0])]
    print ('Loss is', L(variable_vector))
    i -= 1

Loss is 0.017157855027261613
Loss is 0.012227208247171543
Loss is 0.009077916284112217
Loss is 0.006459419488012133
Loss is 0.004043312465918588
Loss is 0.002492173151506337
Loss is 0.0013419683164918657
Loss is 0.0006564899268535414
Loss is 0.00027265027575578603
Loss is 5.5403060777426065e-05
Loss is 5.484248134522269e-06


In [530]:
the_result_vector = fix_vector[['Sigma', 'mu', 'd', 'th', 'th0']].copy()
the_result_vector.loc['minimize_vector'] = [res.x[0], res.x[1], 0, res.x[2], 0]
the_result_vector = the_result_vector*stds+means
the_result_vector.to_pickle('the_result_vectors_to_compare')
the_result_vector

,Sigma,mu,d,th,th0
9030,3.40287e+06,16.6383,89,5.38955,6.5
minimize_vector,1.71722e+06,150,89,2.32542,6.5


In [531]:
[the_result_vector['Sigma'].iloc[0],the_result_vector['Sigma'].iloc[1],the_result_vector['mu'].iloc[0],the_result_vector['mu'].iloc[1],
 the_result_vector['th'].iloc[0],the_result_vector['th'].iloc[1]]

[3402871.538234493,
 1717216.3076813752,
 16.638289933069103,
 149.99999997142905,
 5.389551505199457,
 2.3254157377472593]

## Отошли от нейронки. Занимаемся минимизацией через модельную функцию

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'../solver_/')
from common import tool_times
from common import well_construction as wc
from common.sensor_types import sensors
from solvers.solver_cim_py.interface import Solver


tubes = [
    wc.Tube(d=89., th0=6.5, th=6.5, sigma=3e6, mu=80.),
]

sensor = sensors.PULSE_3E_2
times = tool_times.times[sensor]
solver = Solver(sensor, times, tool_averaging=True)


if __name__ == '__main__':
    output = solver.get_curve(tubes)

    plt.figure()
    plt.semilogy(1e3 * times, output, 'o-')
    plt.ylabel('Signal')
    plt.xlabel('time, ms')
    plt.show()


pass

OSError: /home/alexander/Diploma/solver_/solvers/solver_cim_py/MID/bin/MID2D.dll: invalid ELF header